# Final project

**Name:** Silvia Ivonne Velázquez Ramírez

**e-mail:** silvia.velazquez7499@alumnos.udg.mx

# Modules

In [2]:
#Modules here
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

ModuleNotFoundError: No module named 'ipywidgets'

# Final project

In [ ]:
# Code here